# Calculette Impôts - API Web

Ce repository contient plusieurs applications basées sur la calculatrice M de l'impôt sur le revenu.

In [2]:
# Iportation des biblio
import numpy as np
#import codecs
import math
import configparser # Permet de parser le fichier de paramètres #pip install configparser

Ce projet en python est consacré pour le calcul de l'impot. Comme pour beaucoup, il s'agit d'une tache difficile à comprendre. Ce fut mon cas.
J'ai eu la curiosité de décortiquer tout l processus de calcul des impots.

Commencons par recupérer le barème des impots, le taux d'imposition ainsi que toutes les autres inforations nécessaires

### Le barème progressif de l'impôt sur le revenu 2021
Le barème progressif de l'impôt sur le revenu 2021 est utilisé à partir du 1er janvier 2021 pour calculer l'impôt sur les revenus perçus en 2021. Les tranches du barème ont été revalorisées de 0,2% par rapport au barème 2020. Sont donc imposables en 2021 les personnes qui déclarent plus de 10.084 euros de revenus sur l'année (au lieu de 10 064 euros l'an passé).

Selon le barème 2021, le taux d'imposition applicable aux revenus de 2021 est le suivant :

* Fraction du revenu imposable* inférieure à 10.084 euros : 0%
* Fraction du revenu imposable* de 10.085 euros à 25.710 euros : 11%
* Fraction du revenu imposable* de 25.711 euros à 73.516 euros : 30%
* Fraction du revenu imposable* de 73.517 euros à 158.122 euros : 41%
* Fraction du revenu imposable* supérieure à 158.122 euros : 45%

In [109]:
# -----------------------------------------------------------------------------------------------------------
# Fonction for configuration file
# -----------------------------------------------------------------------------------------------------------
def recup_impots_param(config_file='impot.config'):
    config = configparser.RawConfigParser() # On créé un nouvel objet "config"
    config.read(config_file) # On lit le fichier de paramètres
    # On récupère les valeurs des différents paramètres
    # Récupération basique dans des variables
    annee = config.get(section='DEFAULT',option='annee')
    tranches_impots = config.get(section='Bareme des impots',option='tranches').split(',')
    for i in range(len(tranches_impots)):
        tranches_impots[i] =(int(tranches_impots[i]))
            
    taux_imposition = config.get(section='Bareme des impots',option='taux_imposition').split(',')
    for i in range(len(taux_imposition)):
        taux_imposition[i] =(float(taux_imposition[i]))


    plafond_decote = config.get(section='Reduction',option='plafond_decote').split(',')
    for i in range(len(plafond_decote)):
        plafond_decote[i] =(float(plafond_decote[i]))

    param_decote = config.get(section='Reduction',option='param_decote').split(',')
    for i in range(len(param_decote)):
        param_decote[i] =(float(param_decote[i]))

    ABATTEMENT = config.get(section='Reduction',option='ABATTEMENT').split(',')
    for i in range(len(ABATTEMENT)):
        ABATTEMENT[i] =(float(ABATTEMENT[i]))

    coef_decote = config.getfloat(section='Reduction',option='coef_decote')
    PLAFOND_QF_DEMI_PART = config.getfloat(section='Reduction',option='PLAFOND_QF_DEMI_PART')
    PLAFOND_REVENUS_CELIBATAIRE_POUR_REDUCTION = config.getfloat(section='Reduction',option='PLAFOND_REVENUS_CELIBATAIRE_POUR_REDUCTION')
    PLAFOND_REVENUS_COUPLE_POUR_REDUCTION = config.getfloat(section='Reduction',option='PLAFOND_REVENUS_COUPLE_POUR_REDUCTION')
    VALEUR_REDUC_DEMI_PART = config.getfloat(section='Reduction',option='VALEUR_REDUC_DEMI_PART')

    return annee, tranches_impots, taux_imposition, plafond_decote, param_decote, coef_decote, ABATTEMENT, \
        PLAFOND_QF_DEMI_PART, PLAFOND_REVENUS_CELIBATAIRE_POUR_REDUCTION, PLAFOND_REVENUS_COUPLE_POUR_REDUCTION, VALEUR_REDUC_DEMI_PART
        


In [118]:
annee, tranches_impots, taux_imposition, plafond_decote, param_decote, coef_decote, ABATTEMENT, \
        PLAFOND_QF_DEMI_PART, PLAFOND_REVENUS_CELIBATAIRE_POUR_REDUCTION,\
             PLAFOND_REVENUS_COUPLE_POUR_REDUCTION, VALEUR_REDUC_DEMI_PART = recup_impots_param()
PLAFOND_QF_DEMI_PART


1570.0

![title](https://tahe.developpez.com/tutoriels-cours/python-flask-2020/images/image-78.png)

En entrée, l'algorithme prendra 3 informations

* La sitution famillial  
    choisir : 1. Celibataire, Divorcé, Veuf 2. Couple 3. Autre

* Le nombre d"enfants à charges   

* Le montant des revenu imposable (en euro)

In [95]:
def presentation():
    """
    Fonction de saisie
    """
    # marié : oui, non
    # enfants : nombre d'enfants
    # salaire : salaire annuel
    #print('-- DEBUT de saisie')
    marié = input('Etes-vous marié ? ')
    nb_enfant = int(input("Combien d'enfants avez-vous ?" ))
    assert(nb_enfant >= 0)
    revenu = float(input('\nVeuiller saisir le montant annuel net des salaires: '))
    assert(revenu >= 0)

    marié = marié.strip().lower()
    if (marié == "oui") | (marié == "o") | (marié == "y"):
        nb_parts = nb_enfant*0.5 + 2
        statut_familial='oui'
    else:
        nb_parts = nb_enfant*0.5 + 1
        statut_familial='non'

    # 1 part par enfant à partir du 3ième
    if nb_enfant >= 3:
        # une demi-part de + pour chaque enfant à partir du 3ième
        nb_parts += 0.5 * (nb_enfant - 2)
    print("==> Marié : {} ".format(statut_familial))
    print("==> Nb enfants : {} ".format(nb_enfant))
    print("==> Salaire annuel : {} ".format(revenu))
    print('==> Le nombre de part(s) fiscale est : {0:2.2f}'.format(nb_parts))
    #print('==> Saisie TERMINEE.')
    return statut_familial, nb_enfant,nb_parts, revenu




En contrepartie, pour le calcul de l’impôt dû, le revenu imposable est divisé par le nombre de parts du foyer fiscal à savoir :
* une personne célibataire : 1 part ;
* un couple marié ou pacsé : 2 parts ;
* le 1er enfant à charge : 0,5 parts ;
* le 2ème enfant à charge : 0,5 parts ;
* le 3ème enfant à charge : 1 part ;
* chaque enfant au-delà du 3ème enfant : 1 part.

In [98]:
 # revenu imposable

# revenu_imposable = salaireAnnuel - abattement
# l'abattement a un min et un max
# ----------------------------------------
def get_revenu_imposable(salaire: int) -> int:
    # abattement de 10% du salaire
    abattement = 0.1 * salaire
    # cet abattement ne peut dépasser ABATTEMENT__MAX = ABATTEMENT__MAX[1]
    if abattement > ABATTEMENT[1] :
        abattement = ABATTEMENT[1]

    # l'abattement ne peut être inférieur à ABATTEMENT_MIN = ABATTEMENT__MAX(0)
    if abattement < ABATTEMENT[0]:
        abattement = ABATTEMENT[0]

    # revenu imposable
    revenu_imposable = salaire - abattement
    # quotient familial
    quotient = revenu_imposable / nb_parts
    print('==> Revenu imposable avec déduction 10 % ou frais réels = {0:2.2f} Euros'.format(revenu_imposable))
    print('==> Quotient familial = {0:2.2f} Euros'.format(quotient))
    return revenu_imposable, quotient



In [99]:
print("-- Calcul de l'impot de l'année {} ".format(annee))
marié, nb_enfant, nb_parts, revenu = presentation()
revenu_imposable, qf = get_revenu_imposable(revenu)

-- Calcul de l'impot de l'année 2021 
==> Marié : non 
==> Nb enfants : 0 
==> Salaire annuel : 30000.0 
==> Le nombre de part(s) fiscale est : 1.00
==> Revenu imposable avec déduction 10 % ou frais réels = 27000.00 Euros
==> Quotient familial = 27000.00 Euros


In [155]:

def calcul_impôt(income, tranches_impots, taux_imposition):
    difference = np.diff(tranches_impots)
    state = True
    taxe = 0
    i = 0
    print("-- Application du barème ...")
    while state:
        if income < tranches_impots[i + 1]:
            taxe += (income -
                             tranches_impots[i]) * taux_imposition[i]
            state = False
        else:
            taxe += difference[i] * taux_imposition[i]

            state = True

        print('Impot sur la tranche {} ({}%) = {} Euros'.format(i+1, (taux_imposition[i]*100), taxe))
        i = i + 1
    taux = taux_imposition[i-1]*100  #Le taux marginal d'imposition (TMI)
    return taxe,taux

impot_brute, TMI = calcul_impôt(revenu,tranches_impots, taux_imposition)

print("\n- Avec un taux d'imposition de {}% \n- Impot sur les revenues de l'année est : {} Euros".format(TMI,impot_brute))


-- Application du barème ...
Impot sur la tranche 1 (0.0%) = 0.0 Euros
Impot sur la tranche 2 (11.0%) = 1718.86 Euros
Impot sur la tranche 3 (30.0%) = 3005.8599999999997 Euros

- Avec un taux d'imposition de 30.0% 
- Impot sur les revenues de l'année est : 3005.8599999999997 Euros


### Verification d'une possibilité de réduction d"impot sur le revenu:


![title](https://tahe.developpez.com/tutoriels-cours/python-flask-2020/images/image-80.png)

In [107]:
print('\n-- Verification pour une possibilité de réduction d"impot sur le revenu ...')

# calcule une décôte éventuelle
def get_décôte(marié: str, salaire: int, impots: int) -> int:
    # au départ, une décôte nulle
    décôte = 0
    # montant maximal d'impôt pour avoir la décôte
    plafond_impôt_pour_décôte = plafond_decote[1] if marié == "oui" else plafond_decote[0]
    if impots < plafond_impôt_pour_décôte:
        # montant maximal de la décôte
        plafond_décôte = param_decote[1] if marié == "oui" else param_decote[1]
        # décôte théorique
        décôte = plafond_décôte - coef_decote * impots
        # la décôte ne peut dépasser le montant de l'impôt
        if décôte > impots:
            décôte = impots

        # pas de décôte <0
        if décôte < 0:
            décôte = 0
    print('==> Décote : {0:2.2f} Euros'.format(décôte))
    # résultat
    return décôte
    
# calcul d'une éventuelle décôte    
decote = get_décôte(marié, revenu_imposable, impot_brute)


-- Verification pour une possibilité de réduction d"impot sur le revenu ...
==> Décote : 0.00 Euros


In [114]:
impot_total1 = impot_brute - decote
impot_total1

3005.8599999999997

In [126]:
def calcul_réduction(marié: str, salaire: int, enfants: int, impots: int) -> int:
    """la réduction d’impôt sous condition de revenu fiscal de référence (RFR)"""
    # le plafond des revenus pour avoir droit à la réduction de 20%
    plafond_revenu_pour_réduction = PLAFOND_REVENUS_COUPLE_POUR_REDUCTION if marié == "oui" else PLAFOND_REVENUS_CELIBATAIRE_POUR_REDUCTION
    plafond_revenu_pour_réduction += enfants * VALEUR_REDUC_DEMI_PART
    if enfants > 2:
        plafond_revenu_pour_réduction += (enfants - 2) * VALEUR_REDUC_DEMI_PART

    # réduction
    réduction = 0
    if revenu_imposable < plafond_revenu_pour_réduction:
        # réduction de 20%
        réduction = 0.2 * impots
    print('==> Reduction de l"impot sous condition de RFR : {0:2.2f} Euros'.format(réduction))
    # résultat
    return réduction

# calcul d'une éventuelle réduction d'impôts
réduction = calcul_réduction(marié, revenu_imposable, nb_enfant, impot_total1)

==> Reduction de l"impot sous condition de RFR : 0.00 Euros


In [127]:
impot_total = impot_total1 - réduction
impot_total

3005.8599999999997

In [141]:
print('\nTOTAL D"IMPOSITION NETTE        >> {0:2.2f} Euros <<\n'.format(impot_total))
print("TAUX  D'IMPOSITION PERSONNALISé >> {0:2.2f} % <<\n".format(impot_total * 100 / revenu_imposable))


TOTAL D"IMPOSITION NETTE        >> 3005.86 Euros <<

TAUX  D'IMPOSITION PERSONNALISé >> 11.13 % <<



### Script principal 

In [148]:
from flask import Flask, redirect, url_for, request,render_template  # pip install flask
app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home_page.html')
    
if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\Users\a844026\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [149]:
# app/__init__.py

from flask import Flask, render_template

def create_app():
    app = Flask(__name__)

    @app.route('/')
    def homepage():
        return render_template('homepage.html')

    @app.route('/about/')
    def about():
        return render_template('about.html')

    @app.route('/hello/')
    @app.route('/hello/<name>')
    def hello(name='diallo'):
        return render_template('hello.html', name=name)

    return app

In [147]:
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():

    int_features = [int(x) for x in request.form.values()]
    final_features = [np.array(int_features)]
    
    return render_template('index.html', prediction_text='Sales should be $ {}'.format(10))

@app.route('/results',methods=['POST'])
def results():
    data = request.get_json(force=True)
    return jsonify(output)

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\Users\a844026\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
